# Python Decision Tree
A Jupyter notebook for building a machine learning decision tree.

## 1. Import libraries

## 2. Load data

## 3. Build and fit decision tree

<br><br><br><br><br><br>
## 4. Evaluate decision tree

In [ ]:
import graphviz

dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names = X.columns,  
                         class_names = ["stay","leave"],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

Finally, let's check the accuracy of our tree!